This notebook allows you to run a word alignment assessment and view all the results in their entirety, to help with debugging.

In [ ]:
from pathlib import Path
from typing import Optional, Literal

import modal
from pydantic import BaseModel
import pandas as pd

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
# from app import Assessment

In [ ]:
class Assessment(BaseModel):
    assessment: Optional[int] = None
    revision: int
    reference: int
    type: Literal["word-alignment"]

In [ ]:
! pip install ipywidgets

In [ ]:
stub = modal.Stub(
    name="run-word-alignment-debug",
    image=modal.Image.debian_slim().pip_install(
        "pandas==1.4.3",
        "machine==0.0.1",
        "sil-machine[thot]>=0.8.3",
        "asyncio"
    ).copy(
        mount=modal.Mount.from_local_dir(
            local_path=Path("../../fixtures/"), remote_path=Path("/root/fixtures")
        ),
    ),
)

In [ ]:
stub.run_word_alignment = modal.Function.from_name("word-alignment-test", "assess")

In [ ]:
@stub.function(
            timeout=3600, 
            mounts=[
    *modal.create_package_mounts(["app", "word_alignment_steps"]),
    modal.Mount.from_local_dir(local_path="./", remote_path="/"),
])
def get_results(config):
    from app import Assessment
    assessment_config = Assessment(
            revision=config['revision'], 
            reference=config['reference'], 
            type='word-alignment'
            )
    results = modal.container_app.run_word_alignment.call(assessment_config, return_all_results = True)
    return results

In [ ]:
def assess_draft(revision, reference):
    with stub.run():

        config = {
                'revision':revision, 
                'reference':reference, 
        }
        
        results = get_results.call(config=config)
    return results

In [ ]:
revision = 481 # Isenye
reference = 163 # Hebrew Lemma

In [ ]:
results = assess_draft(revision, reference)

In [ ]:
results.shape

In [ ]:
all_results = results

In [ ]:
all_results.loc[:, 'total_score'] = all_results.apply(lambda row: (row['avg_aligned'] + row['translation_score'] + row['alignment_score'] + row['match_score'] + row['embedding_score']) / 5, axis=1)


In [ ]:
all_results.loc[:, 'total_score'] = all_results[['avg_aligned', 'alignment_score', 'translation_score', 'match_score', 'embedding_score']].mean(axis=1)

In [ ]:
all_results

In [ ]:
results[results['vref'] == 'LUK 1:1']

In [ ]:
total_scores_df = results[['vref', 'source', 'target', 'total_score']]
top_source_scores_df = total_scores_df.fillna(0)
# top_source_scores_df = top_source_scores_df.loc[top_source_scores_df.groupby(['vref', 'source'], sort=False)['total_score'].idxmax(), :].reset_index(drop=True)
top_source_scores_df = top_source_scores_df.loc[top_source_scores_df.groupby(['vref', 'source'], sort=False, as_index=False)

In [ ]:
top_source_scores_df

In [ ]:
top_source_scores_df.loc[top_source_scores_df.groupby(['vref', 'source'], sort=False)['total_score'].idxmax(), :].reset_index(drop=True)

In [ ]:
top_source_scores_df.loc[top_source_scores_df.groupby(['vref', 'source'], sort=False, as_index=False).idxmax()]

In [ ]:
top_source_scores_df[:100]

In [ ]:
verse_scores_df = top_source_scores_df.groupby('vref', as_index=False, sort=False).mean()
verse_scores_df = verse_scores_df.fillna(0)

In [ ]:
verse_scores_df['total_score'].mean()
verse_scores_df.loc[:, 'score'] = verse_scores_df['total_score'].apply(lambda x: 16*x)

In [ ]:
verse_scores_df[:200]